In [1]:
import requests
import pandas as pd
import concurrent.futures
import json
import os

In [2]:
league_id = '366970'
general_endpoint = "https://fantasy.premierleague.com/api/bootstrap-static/"
league_endpoint = f'https://fantasy.premierleague.com/api/leagues-classic/{league_id}/standings'
fixture_endpoint = 'https://fantasy.premierleague.com/api/fixtures/'

In [3]:
def compare_cols(df, col_1, col_2):
    """Compare values in two columns of a pandas DataFrame.

    If col_1 == col_2 return 0, if col_1 > col_2 return 3 else return 0

    Args:
    - df: pandas DataFrame
    - col_1: str, name of the first column to compare
    - col_2: str, name of the second column to compare

    Returns:
    - pandas Series with the comparison result
    """
    # compare values in col_1 and col_2
    result = pd.Series(0, index=df.index)
    result[df[col_1] > df[col_2]] = 3
    result[df[col_1] < df[col_2]] = 0
    result[df[col_1] == df[col_2]] = 1
    return result


def fixture_result(df, col_1, col_2):
    """Compare values in two columns of a pandas DataFrame.

    If col_1 == col_2 return 0, if col_1 > col_2 return 3 else return 0

    Args:
    - df: pandas DataFrame
    - col_1: str, name of the first column to compare
    - col_2: str, name of the second column to compare

    Returns:
    - pandas Series with the comparison result
    """
    # compare values in col_1 and col_2
    result = pd.Series(0, index=df.index)
    result[df[col_1] > df[col_2]] = 'W'
    result[df[col_1] < df[col_2]] = 'L'
    result[df[col_1] == df[col_2]] = 'D'
    return result


def count_categories(df, id_col, cat_col):
    """
    Count the occurrences of each category in a column grouped by ID.

    Parameters:
    df (pandas.DataFrame): the input DataFrame
    id_col (str): the name of the ID column
    cat_col (str): the name of the column to count categories for

    Returns:
    pandas.DataFrame: a DataFrame containing the count of each category
        for each ID
    """
    # group by ID and category, and count the occurrences
    counts = df.groupby([id_col, cat_col])[cat_col].count()

    # convert the counts to a DataFrame and reset the index
    counts = counts.to_frame(name='count').reset_index()

    # pivot the table to have the categories as columns and IDs as rows,
    # and fill in missing values with 0
    counts = counts.pivot(index=id_col, columns=cat_col, values='count').fillna(0)

    return counts

In [4]:
def fetch_data(general_endpoint, league_endpoint, fixture_endpoint):
    with requests.Session() as session:
        general_response = session.get(general_endpoint).json()
        league_response = session.get(league_endpoint).json()
        fixture_response = session.get(fixture_endpoint).json()

    real_player_data = pd.DataFrame.from_records(general_response['elements'], columns=['id', 'web_name', 'selected_by_percent', 'total_points', 'bonus', 'form', 'value_season', 'value_form' ]).astype({"form": float, "value_season": float, "value_form": float}).set_index('id')
    gameweek_data = pd.DataFrame.from_records(general_response['events'], columns=['id', "deadline_time",'is_current', 'most_captained', 'most_vice_captained', 'top_element']).fillna(0).astype({"most_captained": int, "most_vice_captained": int, "top_element": int}).set_index('id')
    
    real_team_names = pd.DataFrame.from_records(general_response['teams'], columns=['id', 'name'])
    fixture_ids = pd.DataFrame.from_records(fixture_response, columns =['event', 'team_a', 'team_h'])
    manager_ids = pd.DataFrame.from_records(league_response['standings']['results'], columns=['id', 'entry', "rank", 'player_name']).astype({"rank": int}).set_index('id')
    current_week = gameweek_data.index[gameweek_data['is_current']].item()
    
    current_standings = pd.DataFrame.from_records(league_response['standings']['results'], columns=['id', 'entry', 'player_name', 'rank', 'last_rank', 'total']).astype({"rank": int, "last_rank": int, "total": int}).set_index('entry')
    current_week = gameweek_data.index[gameweek_data['is_current']].item()
    
    manager_teams = pd.concat([pd.DataFrame.from_records(session.get(f'https://fantasy.premierleague.com/api/entry/{i}/event/{current_week}/picks/').json()['picks'], ).assign(entry=i) for i in current_standings.index])
    current_gameweek_teams = pd.merge(manager_teams, real_player_data, left_on='element', right_index=True)

    template_teams = current_gameweek_teams.astype({"selected_by_percent": float}).groupby('entry')['selected_by_percent'].mean().sort_values(ascending=False).to_frame(name='selected_by_percent')
    template_teams = current_standings.join(template_teams, on='entry')[['player_name', 'rank', 'total', 'selected_by_percent']]


    # JSON string with team names and ids
    json_str = real_team_names.to_json(orient='records')

    # Convert JSON string to dictionary
    team_dict = json.loads(json_str)

    # Create a dictionary of team IDs and names
    team_id_name_dict = {team['id']: team['name'] for team in team_dict}

    fixture_ids['team_a_name'] = fixture_ids['team_a'].map(team_id_name_dict)
    fixture_ids['team_h_name'] = fixture_ids['team_h'].map(team_id_name_dict)
    
    fixture_teams_file_path = 'fixture_teams.csv'
    if not os.path.isfile(fixture_teams_file_path):
        permanent_fixture_team = manager_ids
        permanent_fixture_team['fixture_team'] = permanent_fixture_team['rank'].map(team_id_name_dict)
        permanent_fixture_team.to_csv(fixture_teams_file_path, index=False)
    else:
        permanent_fixture_team = pd.read_csv(fixture_teams_file_path)


    # JSON string with team names and ids
    json_str2 = permanent_fixture_team.to_json(orient='records')

    # Convert JSON string to dictionary
    team_dict2 = json.loads(json_str2)

    # Create a dictionary of team IDs and names
    team_id_name_dict2 = {team['fixture_team']: team['player_name'] for team in team_dict2}

    fixture_ids['player_a_name'] = fixture_ids['team_a_name'].map(team_id_name_dict2)
    fixture_ids['player_h_name'] = fixture_ids['team_h_name'].map(team_id_name_dict2)



    # JSON string with team names and ids
    json_str3 = permanent_fixture_team.to_json(orient='records')

    # Convert JSON string to dictionary
    team_dict3 = json.loads(json_str3)

    # Create a dictionary of team IDs and names
    team_id_name_dict3 = {team['player_name']: team['entry'] for team in team_dict3}

    fixture_ids['player_a_id'] = fixture_ids['player_a_name'].map(team_id_name_dict3)
    fixture_ids['player_h_id'] = fixture_ids['player_h_name'].map(team_id_name_dict3)
    fixture_ids = fixture_ids.fillna(0).astype({"player_a_id": int, "player_h_id": int})


    history_data = pd.DataFrame()
    for i in permanent_fixture_team.entry:
        history_endpoint = f'https://fantasy.premierleague.com/api/entry/{i}/history/'
        history_response = requests.get(history_endpoint).json()
        temp_data = pd.DataFrame.from_records(history_response['current'], columns=['event', 'points', 'event_transfers_cost'])
        temp_data['h2h_points'] = temp_data['points'] - temp_data['event_transfers_cost']
        temp_data['entry'] = i
        if history_data.shape == (0,0):
            history_data = temp_data
        else:
            history_data = pd.concat([history_data, temp_data])


    final = fixture_ids.merge(history_data, left_on=['event', 'player_a_id'], right_on=['event',"entry"])
    final = final.merge(history_data, left_on=['event', 'player_h_id'], right_on=['event',"entry"])
    final= final.iloc[:, [0,1,2,3,4,5,6,7,8,11,15]]
    final.rename(columns = {'h2h_points_x':'h2h_gw_points_a', 'h2h_points_y':'h2h_gw_points_h'}, inplace = True)
    final['h2h_table_points_a'] = compare_cols(final, 'h2h_gw_points_a', 'h2h_gw_points_h')
    final['h2h_table_points_h'] = compare_cols(final, 'h2h_gw_points_h', 'h2h_gw_points_a')

    final['h2h_table_results_a'] = fixture_result(final, 'h2h_gw_points_a', 'h2h_gw_points_h')
    final['h2h_table_results_h'] = fixture_result(final, 'h2h_gw_points_h', 'h2h_gw_points_a')

    away_pts = final.groupby('player_a_name')['h2h_table_points_a'].sum()
    home_pts = final.groupby('player_h_name')['h2h_table_points_h'].sum()
    h2h_table = pd.concat([away_pts, home_pts], axis=1)
    h2h_table.columns = ['away_pts', 'home_pts']
    h2h_table['total_pts'] = h2h_table['away_pts'] + h2h_table['home_pts']
    h2h_table = h2h_table.sort_values(by='total_pts', ascending=False).rename_axis('player_name')


    away_fix = count_categories(final, 'player_a_name', 'h2h_table_results_a').astype({"D": int,"L": int,"W": int}).rename_axis('player_name')
    home_fix = count_categories(final, 'player_h_name', 'h2h_table_results_h').astype({"D": int,"L": int,"W": int}).rename_axis('player_name')
    total_fix = away_fix.join(home_fix, lsuffix='_away', rsuffix='_home')

    total_fix['W'] = total_fix['W_away'] + total_fix['W_home']
    total_fix['L'] = total_fix['L_away'] + total_fix['L_home']
    total_fix['D'] = total_fix['D_away'] + total_fix['D_home']
    total_fix =  total_fix.iloc[:, [6,7,8]]

    h2h_table = h2h_table.merge(total_fix, on='player_name')
    h2h_table['played'] = h2h_table['W'] +h2h_table['L']+ h2h_table['D']
    h2h_table = h2h_table.iloc[:, [2,3,4,5,6]]
    h2h_table = h2h_table[['played', 'W', 'D', 'L', 'total_pts']]

    

    return gameweek_data, current_standings, template_teams, fixture_ids, permanent_fixture_team, h2h_table

In [5]:
gameweek_data, current_standings, template_teams, fixture_ids, permanent_fixture_team, h2h_table= fetch_data(general_endpoint, league_endpoint,fixture_endpoint)

In [6]:
#Returns the deadline of the GW, the most captained and vice captained player_ids and the highest scoring player_id
gameweek_data.head()

,deadline_time,is_current,most_captained,most_vice_captained,top_element
id,,,,,
1,2022-08-05T17:30:00Z,False,283,318,366
2,2022-08-13T10:00:00Z,False,318,283,28
3,2022-08-20T10:00:00Z,False,28,318,7
4,2022-08-27T10:00:00Z,False,283,28,279
5,2022-08-30T17:00:00Z,False,318,28,318


In [7]:
#returns the league standings given a specific league ID
current_standings.head()

,id,player_name,rank,last_rank,total
entry,,,,,
2208836,54837678,Divyam Dixit,1,1,2139
552345,11604984,Marco Gouveia,2,2,2109
534613,11045904,Connor McDonald,3,3,2049
560769,11044918,Alex Wietzorrek,4,4,2047
578302,42389877,Ryan Shacks,5,5,2029


In [8]:
#shows the average 'selected_by' value for each team in the league, indicator of 'templateness'
template_teams.head()

,player_name,rank,total,selected_by_percent
entry,,,,
2208836,Divyam Dixit,1,2139,19.160000
552345,Marco Gouveia,2,2109,23.860000
534613,Connor McDonald,3,2049,25.980000
560769,Alex Wietzorrek,4,2047,25.866667
578302,Ryan Shacks,5,2029,28.620000


In [9]:
#fixtures for head to head league
fixture_ids.head()

,event,team_a,team_h,team_a_name,team_h_name,player_a_name,player_h_name,player_a_id,player_h_id
0,1,1,7,Arsenal,Crystal Palace,Divyam Dixit,Niklas Wietzorrek,2208836,1742534
1,1,12,9,Liverpool,Fulham,Jason Perry,Devon Jansen,3648627,1789073
2,1,2,3,Aston Villa,Bournemouth,Marco Gouveia,Connor McDonald,552345,534613
3,1,20,11,Wolves,Leeds,0,Cole Floyd,0,1947717
4,1,16,15,Nott'm Forest,Newcastle,0,0,0,0


In [10]:
#A permanent team assigned to each league member to determine their head to head
permanent_fixture_team

,entry,rank,player_name,fixture_team
0,2208836,1,Divyam Dixit,Arsenal
1,552345,2,Marco Gouveia,Aston Villa
2,534613,3,Connor McDonald,Bournemouth
3,560769,4,Alex Wietzorrek,Brentford
4,578302,5,Ryan Shacks,Brighton
5,2123591,6,Peter Wertz,Chelsea
6,1742534,7,Niklas Wietzorrek,Crystal Palace
7,1002132,8,Devon Hodgson,Everton
8,1789073,9,Devon Jansen,Fulham
9,1930701,10,Liam Smorfitt,Leicester


In [11]:
h2h_table

,played,W,D,L,total_pts
player_name,,,,,
Divyam Dixit,23,18,1,4,55
Marco Gouveia,22,17,1,4,52
Alex Wietzorrek,22,14,0,8,42
Peter Wertz,22,12,1,9,37
Devon Jansen,22,11,2,9,35
Ryan Shacks,20,10,2,8,32
Connor McDonald,21,10,1,10,31
Niklas Wietzorrek,22,10,0,12,30
Cole Floyd,23,8,1,14,25
